In [ ]:
!git clone https://github.com/abewley/sort.git

ERROR: Could not find a version that satisfies the requirement sort (from versions: none)
ERROR: No matching distribution found for sort
Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 208 (delta 2), reused 1 (delta 1), pack-reused 203 (from 2)
Receiving objects: 100% (208/208), 1.20 MiB | 1.76 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
import torch
from torchvision import transforms
from PIL import Image

In [30]:
import numpy as np
from ultralytics import YOLO
import cv2
import math
from sort.sort import *

cap = cv2.VideoCapture("input_auto_cycle.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_auto.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

model = YOLO("yolov8l.pt", verbose=False)

# 클래스 목록 생략
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Tracking
tracker = Sort(max_age=60, min_hits=3, iou_threshold=0.1)

limitsUp = [500, 350, 900, 350]
limitsDown = [450, 450, 1000, 450]


totalCountUp = []
totalCountDown = []
last_line_result = []
last_line_crossed = {}

# 배경 포함 텍스트 출력 함수
def put_text_with_background(img, text, org, font=cv2.FONT_HERSHEY_SIMPLEX,
                             scale=0.7, text_color=(255, 255, 255), bg_color=(0, 0, 0),
                             thickness=2, padding=4):
    (text_w, text_h), _ = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    cv2.rectangle(img, (x - padding, y - text_h - padding),
                  (x + text_w + padding, y + padding), bg_color, -1)
    cv2.putText(img, text, (x, y), font, scale, text_color, thickness, cv2.LINE_AA)

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)
    detections = np.empty((0, 5))

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass == "person" and conf > 0.2:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    resultsTracker = tracker.update(detections)

    cv2.line(img, tuple(limitsUp[:2]), tuple(limitsUp[2:]), (0, 100, 255), 2, cv2.LINE_AA)
    cv2.line(img, tuple(limitsDown[:2]), tuple(limitsDown[2:]), (0, 0, 255), 2, cv2.LINE_AA)

    for result in resultsTracker:
        x1, y1, x2, y2, id = map(int, result)
        w, h = x2 - x1, y2 - y1
        cx, cy = x1 + w // 2, y1 + h // 2

        # 바운딩 박스
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), 2)
        put_text_with_background(img, f'ID {int(id)}', (x1, y1 - 10))

        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # 선 통과 감지
        if limitsDown[0] < cx < limitsDown[2] and abs(cy - limitsDown[1]) < 15:
            if id in last_line_crossed and last_line_crossed[id] == 'up':
                totalCountDown.append(id)
            if id not in last_line_crossed or last_line_crossed[id] == 'up':
                last_line_crossed[id] = 'down'
                last_line_result.append(last_line_crossed)

        if limitsUp[0] < cx < limitsUp[2] and abs(cy - limitsUp[1]) < 15:
            if id in last_line_crossed and last_line_crossed[id] == 'down':
                totalCountUp.append(id)
            if id not in last_line_crossed or last_line_crossed[id] == 'down':
                
                last_line_crossed[id] = 'up'
                last_line_result.append(last_line_crossed)

    # UI 박스
    cv2.rectangle(img, (30, 30), (330, 130), (0, 0, 0), -1)
    cv2.putText(img, f"Up Count: {len(totalCountUp)}", (45, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 255), 2)
    cv2.putText(img, f"Down Count: {len(totalCountDown)}", (45, 115), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 255), 2)

    out.write(img)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

print(last_line_result)

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 1 car, 1 stop sign, 2 benchs, 189.2ms
Speed: 1.8ms preprocess, 189.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 stop sign, 1 bench, 199.0ms
Speed: 4.6ms preprocess, 199.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 stop sign, 1 bench, 237.6ms
Speed: 1.1ms preprocess, 237.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 stop sign, 1 bench, 231.4ms
Speed: 1.1ms preprocess, 231.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 stop sign, 1 bench, 206.2ms
Speed: 1.3ms preprocess, 206.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 stop sign, 187.4ms
Speed: 1.3ms preprocess, 187.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 stop sign, 1 bench, 186.7ms
Speed: 